In [1]:
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, 'src/')
import ops

sess = tf.InteractiveSession()

voxels_name = glob.glob('data/chairs_voxel/*.npy')
voxels_name = np.array(voxels_name)
voxels = tf.cast(ops.load_voxelbatch(voxels_name[0:64]), 'float32')

In [2]:
def rot(s):
    i = tf.cast((s+1)*2, 'int32')
    vp = tf.constant([0, np.pi/2.0, np.pi, 3*np.pi/2.0])
    
    theta = tf.gather(vp, i)
    phi = 0.0
    
    sin_theta = tf.sin(theta)
    cos_theta = tf.cos(theta)
    sin_phi = tf.sin(phi)
    cos_phi = tf.cos(phi)
    
    ry = [[cos_theta, 0, sin_theta], [0, 1, 0], [-sin_theta, 0, cos_theta]]
    rx = [[1, 0, 0], [0, cos_phi, -sin_phi], [0, sin_phi, cos_phi]]
    
    return tf.matmul(rx, ry)
    #return ry

def flatten(t): return tf.reshape(t, (1, -1))

In [13]:
'''This is a hack. One day tensorflow will have gather_nd properly implemented with backprop.
Until then, use this function'''
def gather_nd(params, indices, name=None):
    shape = params.get_shape().as_list()
    rank = len(shape)
    flat_params = tf.reshape(params, [-1])
    multipliers = [reduce(lambda x, y: x*y, shape[i+1:], 1) for i in range(0, rank)]
    indices_unpacked = tf.unpack(tf.transpose(indices, [rank - 1] + range(0, rank - 1), name))
    flat_indices = sum([a*b for a,b in zip(multipliers, indices_unpacked)])
    return tf.gather(flat_params, flat_indices, name=name)

def grid_coord(h, w, d):
    xl = tf.linspace(-1.0, 1.0, w)
    yl = tf.linspace(-1.0, 1.0, h)
    zl = tf.linspace(-1.0, 1.0, d)
    
    xs, ys, zs = tf.meshgrid(xl, yl, zl, indexing='ij')
    g = tf.concat(0,[flatten(xs), flatten(ys), flatten(zs)])
    return g

'''
grid = grid_coord(32, 32, 32)
xs = (grid[0, :] + 1.0) * 32 / 2.0
ys = (grid[1, :] + 1.0) * 32 / 2.0
zs = (grid[2, :] + 1.0) * 32 / 2.0
idxs = tf.cast(tf.transpose(tf.pack([xs, ys, zs])), 'int32')
idxs = tf.expand_dims(idxs, 0)
v = tf.constant(voxels[0])
print tf.reduce_sum(v).eval()
print tf.reduce_sum(tf.abs(v-tf.reshape(gather_nd(v, idxs), [32, 32, 32]))).eval()
''' 

def get_voxel_values(v, xs, ys, zs):
    idxs = tf.cast(tf.pack([xs, ys, zs], axis=1), 'int32')
    idxs = tf.clip_by_value(idxs, 0, v.get_shape()[0])
    idxs = tf.expand_dims(idxs, 0)
    return gather_nd(v, idxs)

def resample_voxels(v, xs, ys, zs, method="trilinear"):
    
    if method == "trilinear":
        floor_xs = tf.floor(tf.clip_by_value(xs, 0, 64))
        floor_ys = tf.floor(tf.clip_by_value(ys, 0, 64))
        floor_zs = tf.floor(tf.clip_by_value(zs, 0, 64))

        ceil_xs = tf.ceil(tf.clip_by_value(xs, 0, 64))
        ceil_ys = tf.ceil(tf.clip_by_value(ys, 0, 64))
        ceil_zs = tf.ceil(tf.clip_by_value(zs, 0, 64))

        final_value =( tf.abs((xs-floor_xs)*(ys-floor_ys)*(zs-floor_zs))*get_voxel_values(v, ceil_xs, ceil_ys, ceil_zs) + 
                       tf.abs((xs-floor_xs)*(ys-floor_ys)*(zs-ceil_zs))*get_voxel_values(v, ceil_xs, ceil_ys, floor_zs) +
                       tf.abs((xs-floor_xs)*(ys-ceil_ys)*(zs-floor_zs))*get_voxel_values(v, ceil_xs, floor_ys, ceil_zs) +
                       tf.abs((xs-floor_xs)*(ys-ceil_ys)*(zs-ceil_zs))*get_voxel_values(v, ceil_xs, floor_ys, floor_zs) +
                       tf.abs((xs-ceil_xs)*(ys-floor_ys)*(zs-floor_zs))*get_voxel_values(v, floor_xs, ceil_ys, ceil_zs) +
                       tf.abs((xs-ceil_xs)*(ys-floor_ys)*(zs-ceil_zs))*get_voxel_values(v, floor_xs, ceil_ys, floor_zs) +
                       tf.abs((xs-ceil_xs)*(ys-ceil_ys)*(zs-floor_zs))*get_voxel_values(v, floor_xs, floor_ys, ceil_zs) +
                       tf.abs((xs-ceil_xs)*(ys-ceil_ys)*(zs-ceil_zs))*get_voxel_values(v, floor_xs, floor_ys, floor_zs)
                     )
        return final_value
    
    elif method == "nearest":
        r_xs = tf.round(xs)
        r_ys = tf.round(ys)
        r_zs = tf.round(zs)
        return get_voxel_values(v, r_xs, r_ys, r_zs)
    
    else:
        raise NameError(method)
    
def transform(v, t):
    height = int(v.get_shape()[0])
    width = int(v.get_shape()[1])
    depth = int(v.get_shape()[2])
    grid = grid_coord(height, width, depth)
    
    xs = grid[0, :]
    ys = grid[1, :]
    zs = grid[2, :]
    
    idxs_f = tf.transpose(tf.pack([xs, ys, zs]))
    idxs_f = tf.matmul(idxs_f, t)
    
    xs_t = (idxs_f[:, 0] + 1.0) * float(width) / 2.0
    ys_t = (idxs_f[:, 1] + 1.0) * float(height) / 2.0
    zs_t = (idxs_f[:, 2] + 1.0) * float(depth) / 2.0

    idxs = tf.cast(tf.pack([xs_t, ys_t, zs_t], axis=1), 'int32')
    idxs = tf.clip_by_value(idxs, 0, v.get_shape()[0])
    idxs = tf.expand_dims(idxs, 0)
    
    return tf.reshape(resample_voxels(v, xs_t, ys_t, zs_t, method='trilinear'), v.get_shape())

def project(v, tau=1):
    p = tf.reduce_sum(v, 2)
    p = tf.ones_like(p) - tf.exp(-p*10)
    return tf.reverse(tf.transpose(p), [True, False])

'''
#xs = (grid[0, :] + 1.0) * 32 / 2.0
#ys = (grid[1, :] + 1.0) * 32 / 2.0
#zs = (grid[2, :] + 1.0) * 32 / 2.0
idxs_f = tf.transpose(tf.pack([xs, ys, zs]))
idxs_f = tf.matmul(idxs_f, rot(0.0, -np.pi/8.0))

xs_t = (idxs_f[:, 0] + 1.0) * 32 / 2.0
ys_t = (idxs_f[:, 1] + 1.0) * 32 / 2.0
zs_t = (idxs_f[:, 2] + 1.0) * 32 / 2.0

idxs = tf.cast(tf.pack([xs_t, ys_t, zs_t], axis=1), 'int32')
idxs = tf.clip_by_value(idxs, 0, 32)
idxs = tf.expand_dims(idxs, 0)
v = tf.constant(voxels[0])
v_t = tf.reshape(gather_nd(v, idxs), [32, 32, 32])
'''

v = voxels[3]
np.save("original.npy", transform(v, rot(np.random.uniform(-1, 1))).eval())
np.save("transformed.npy", transform(v, rot(np.random.uniform(-1, 1))).eval())

plt.imshow(project(transform(v, rot(np.random.uniform(-1, 1)))).eval(), cmap='gray', interpolation='none')
#plt.imshow(project(v).eval(), cmap='gray', interpolation='none')
plt.show()